In [ ]:
# build module
!python "../openstorm_radar_py/setup.py" build -j 8

In [ ]:
import sys
import os
import numpy as np
import time
import math
import matplotlib.animation
import IPython
from matplotlib import pyplot as plt
import matplotlib

#%config InlineBackend.figure_formats = ['svg']
#%config InlineBackend.figure_formats = ['jpeg']
%config InlineBackend.figure_formats = ['retina']
#%config InlineBackend.figure_formats = ['png']
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["figure.figsize"] = (7, 7)


sys.path.insert(0, '../')
import openstorm_radar_py



In [ ]:
# create RadarDataHolder object
radar_data_holder = openstorm_radar_py.RadarDataHolder()

In [ ]:
# select products to be loaded
reflectivity_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_REFLECTIVITY)
velocity_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_VELOCITY)
# derived products
srv_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_STORM_RELATIVE_VELOCITY)
rotation_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_ROTATION)

In [ ]:
# load file
radar_data_holder.load("./data/Radar/l2data/KTLX20130531_231434_V06.gz")
#radar_data_holder.load("../OpenStorm/Content/Data/Demo/KTLX20130531_231434_V06")
#radar_data_holder.load("../files/el-reno/compressed/KTLX20130531_231434_V06.gz")
# wait for it to finish loading because RadarDataHolder is asynchronous and multi-threaded
while(radar_data_holder.get_state() == openstorm_radar_py.RadarDataHolder.DataStateLoading):
	print("loading...", end='\r')
	time.sleep(0.1)
print("loaded      ", end='\n')

In [ ]:
# get now loaded RadarData objects from products
print("is reflectivity loaded?", reflectivity_product.is_loaded(), "   is velocity loaded?", velocity_product.is_loaded(), "   is srv loaded?", srv_product.is_loaded(), "   is rotation loaded?", rotation_product.is_loaded())
reflectivity_data = reflectivity_product.get_radar_data()
velocity_data = velocity_product.get_radar_data()
srv_data = srv_product.get_radar_data()
rotation_data = rotation_product.get_radar_data()
velocity_data.get_stats()

In [ ]:
# a function to correctly plot sweep buffers
def plot_radial_image(buffer):
	theta, rad = np.meshgrid(np.linspace(np.pi * 2.5, np.pi * 0.5, buffer.shape[0]), np.linspace(0, buffer.shape[1], buffer.shape[1]))
	#print(theta.shape, rad.shape)
	fig = plt.figure()
	ax = fig.add_subplot(111, polar='True')
	ax.pcolormesh(theta, rad, np.transpose(buffer), shading='auto')
	plt.show()

In [ ]:
print(reflectivity_data._ptr, velocity_data._ptr, rotation_data._ptr, srv_data._ptr)
print(reflectivity_data.buffer)
#time.sleep(1)
plot_radial_image(np.array(reflectivity_data.buffer)[0,1:-1,:1000])
#time.sleep(0.5)
plot_radial_image(np.array(velocity_data.buffer)[0,1:-1,:1000])
#time.sleep(0.5)
plot_radial_image(np.clip(np.array(srv_data.buffer),-10, 10)[0,1:-1,:1000])
#time.sleep(0.5)
plot_radial_image(np.array(rotation_data.buffer)[0,1:-1,:1000])

In [ ]:
import importlib
import tornado_data
importlib.reload(tornado_data)

# preload data before timing
tornado_data.get_all_tornados()
# test generating the mask of where tornados are
start_time = time.time()
mask = tornado_data.generate_mask(srv_data)[0][:,:1000]
end_time = time.time()
print("generating mask took", end_time - start_time)

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot()
ax.imshow(mask, interpolation='nearest')
plt.show()
plot_radial_image(mask)

In [ ]:
# make an animation out of the files in a directory and plot tornados as red dots using tornado_data.py
if 0:
	print("start animation", end='\r')
	fig = plt.figure(figsize=(12,12))
	ax = fig.add_subplot(111, polar='True')

	#animation_data_dir = "../files/el-reno/"
	#animation_data_dir = "../files/tornado-2011/"
	animation_data_dir = "data/Radar/l2data/"
	animation_data_files = list(map(lambda x: animation_data_dir + x, os.listdir(animation_data_dir)))
	animation_data_files.sort()
	animation_data_files = animation_data_files[2:]

	def drawframe(frame):
		radius = 1000
		print("frame", frame + 1, "             ", end='\r')
		time.sleep(0.05)
		
		radar_data_holder = openstorm_radar_py.RadarDataHolder()
		
		ref_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_REFLECTIVITY)
		#rot_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_ROTATION)
		srv_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_STORM_RELATIVE_VELOCITY)
		
		radar_data_holder.load(animation_data_files[frame])
		# wait for it to finish loading because RadarDataHolder is asynchronous and multi-threaded
		while(radar_data_holder.get_state() == openstorm_radar_py.RadarDataHolder.DataStateLoading):
			time.sleep(0.1)
		if srv_product.is_loaded() == False:
			return tuple([])
			
		radar_data = ref_product.get_radar_data()
		#radar_data = rot_product.get_radar_data()
		buffer = np.array(radar_data.buffer)[0,1:-1,:radius]
		
		#radar_data = srv_product.get_radar_data()
		#buffer = np.clip(np.array(radar_data.buffer),-20, 20)[0,1:-1,:radius]
		
		print("frame", frame + 1, "loaded       ", end='\r')
		#buffer = np.array(radar_data.buffer)[0,1:-1,:1000]
		theta, rad = np.meshgrid(np.linspace(np.pi * 2.5, np.pi * 0.5, buffer.shape[0]), np.linspace(0, 1, buffer.shape[1]))
		ax.clear()
		ar = ax.pcolormesh(theta, rad, np.transpose(buffer), shading='auto')
		# plot mask overlay
		transparent_cmap = matplotlib.colors.LinearSegmentedColormap.from_list('transparent_cmap',["white","red"],256)
		transparent_cmap._init()
		alphas = np.linspace(0, 0.3, transparent_cmap.N+3)
		transparent_cmap._lut[:,-1] = alphas
		mask, tornado_info = tornado_data.generate_mask(radar_data)
		mask = mask[:,:radius]
		ar2 = ax.pcolormesh(theta, rad, np.transpose(mask), shading='auto', cmap=transparent_cmap)
		return tuple([ar, ar2])
	anim = matplotlib.animation.FuncAnimation(fig, drawframe, frames=len(animation_data_files), interval=1000/10, blit=True)
	writer_video = matplotlib.animation.FFMpegWriter(fps=1000. / anim._interval)
	anim.save('test_animation.mp4', writer=writer_video)
	# IPython.display.display(IPython.display.HTML(anim.to_html5_video()))
	plt.close()

In [ ]:
import time
import math
import multiprocessing
from matplotlib import pyplot as plt
import numpy as np
import importlib
import dataset
importlib.reload(dataset)

dataset_files = dataset.DirectoryTrainTest("./data/Radar/l2data", train_percentage=90)
print("Split", len(dataset_files.train_list), len(dataset_files.test_list))
thread_count = max(math.ceil(multiprocessing.cpu_count() / 2) - 2, 1)
tornado_dataset = dataset.TornadoDataset(dataset_files.test_list, thread_count=8, buffer_size=16, section_size=256, cache_results=False)
#time.sleep(10)
for i in range(10):
	data = tornado_dataset.next()
	#print(data)
	print(data["data"].shape, data["mask"].shape, data["file"], data["bounds"])
	def normalize_array(x):
		non_inf = x[np.logical_and(x != -np.inf, x != np.inf)]
		if non_inf.shape[0] == 0:
			non_inf = np.array([0])
		return (x-np.min(non_inf))/(np.max(non_inf)-np.min(non_inf))
	image = np.stack([
		normalize_array(data["data"][0,0,:,:]), # reflectivity
		#normalize_array(data["data"][0,:,:,1]), # relative storm velocity
		data["mask"] / 2,
		normalize_array(data["data"][2,0,:,:]),	# spectrum width
	], axis=-1)
	plt.imshow(image, interpolation='nearest')
	plt.show()
	for i in range(5):
		tornado_dataset.next()
tornado_dataset.destroy()

In [ ]:
import torch
import torchvision.utils
import torch.utils.data
import importlib
import dataset
importlib.reload(dataset)

tornado_dataset = dataset.TornadoDataset(dataset_files.test_list, thread_count=4, buffer_size=4, section_size=256, auto_shuffle=True)
torch_tornado_dataset = dataset.TorchDataset(tornado_dataset)
data_loader = torch.utils.data.DataLoader(torch_tornado_dataset, 16) #, num_workers=2
data_iter = iter(data_loader)
item = next(data_iter)
print(item["data"].shape, item["mask"].shape)
def matplotlib_imshow(img, one_channel=False):
	if one_channel:
		img = img.mean(dim=0)
	#img = img / 2 + 0.5     # unnormalize
	
	npimg = img.cpu().numpy()
	if one_channel:
		plt.imshow(npimg, cmap="Greys")
	else:
		plt.imshow(np.transpose(npimg, (1, 2, 0)))
	plt.show()
img_grid = torchvision.utils.make_grid(item["data"][:,:3,0], nrow=4)
matplotlib_imshow(img_grid, one_channel=False)
img_grid = torchvision.utils.make_grid(item["data"][:,:3,0] / 2 + 0.5, nrow=4)
matplotlib_imshow(img_grid, one_channel=False)
tornado_dataset.destroy()


In [ ]:
img_grid = torchvision.utils.make_grid(item["data"][:,[0,1,3],0] / 2 + 0.5, nrow=4)
matplotlib_imshow(img_grid, one_channel=False)

In [ ]:
import importlib
import model
importlib.reload(model)
tornado_detection_model = model.TornadoDetectionModel()
pytorch_total_params = sum(p.numel() for p in tornado_detection_model.parameters())
print("parameter count", pytorch_total_params)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if os.name == 'nt':
	pass
	#device = torch.device("cpu")
input_data = item["data"]
input_data = input_data.to(device, non_blocking=False)
tornado_detection_model.to(device, non_blocking=False)
for i in range(5):
	print("run", i)
	output = tornado_detection_model(input_data)
	print(torch.mean(output).cpu())
# print("start")
# output = tornado_detection_model(input_data)
# print("start2")
# output = tornado_detection_model(input_data)
print("output", output.shape)
torch.cuda.empty_cache()

In [ ]:
# this should show some progress or something is very wrong
import torch.nn.functional
importlib.reload(model)
tornado_detection_model = model.TornadoDetectionModel()
tornado_detection_model.to(device)
#optimizer = torch.optim.SGD(tornado_detection_model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.AdamW(tornado_detection_model.parameters(),lr=0.0001)

loss_function = model.MaskLoss()
loss_function.to(device)


input_data = item["data"]
input_data = input_data.to(device)
mask = item["mask"]
mask = mask.to(device)

for step in range(1000):
	
	optimizer.zero_grad()
	
	output = tornado_detection_model(input_data)
	#loss = torch.mean(torch.maximum(1 - output, torch.tensor(0)))
	loss, extra_loss_info = loss_function(output, mask)
	
	loss.backward()
	optimizer.step()
	
	tornado_detection_model.train(False)
	mean_out = torch.mean(output)
	min_out = torch.min(output)
	max_out = torch.max(output)
	print("step", step, "loss", loss.item(), "mean", mean_out.detach().cpu().numpy(), "min", min_out.detach().cpu().numpy(), "max", max_out.detach().cpu().numpy())
	print("outside_mask", extra_loss_info["outside_mask"].detach().cpu().numpy())
	print("inside_mask", extra_loss_info["inside_mask"].detach().cpu().numpy())
	images = torch.stack([
		mask,
		output,
		input_data[:,0,0,:,:],
	], 1)
	images = torch.nn.functional.max_pool2d(images, 4)
	img_grid = torchvision.utils.make_grid(images, nrow=4)
	matplotlib_imshow(img_grid, one_channel=False)
	tornado_detection_model.train(True)
	
torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()